In [ ]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
import mediapipe as mp
import os
from scipy.signal import find_peaks, argrelextrema
from moviepy.video.io.VideoFileClip import VideoFileClip

import matplotlib.pyplot as plt

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Read videos from the folder
# Get the video file names

video_files = os.listdir('videos')
video_files = [file for file in video_files if file.endswith('.mp4')]
video_files = [video_files[2], video_files[3]]
print(video_files)

for file in video_files:
    video = VideoFileClip('videos/' + file)
    duration = video.duration

    middle_start = duration / 2 - 5
    middle_end = duration / 2 + 5

    middle_clip = video.subclip(middle_start, middle_end)

    middle_clip.write_videofile("processed/" + file, audio=False, codec="libx264")



caps = []
for file in video_files:
    cap = cv2.VideoCapture('processed/' + file)
    if not cap.isOpened():
        print("Error opening video stream or file" + file)
    else:
        caps.append(cap)

In [ ]:
def _calculate_angle(p1, p2, p3):
    '''
    Calculate the angle between three points. As defined in report.
    '''
    a = np.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)
    b = np.sqrt((p2[0] - p3[0])**2 + (p2[1] - p3[1])**2)
    c = np.sqrt((p3[0] - p1[0])**2 + (p3[1] - p1[1])**2)
    angle = math.acos((b**2 + c**2 - a**2)/(2*b*c))
    return np.degrees(angle)

In [ ]:
# Calculate lowest y value for each video by looking at local minima of ankle facing the camera
# Filter out innaccuracies by adding condition that peaks should be separated by at least 10 frames
# get the frame number of the lowest y value for each video

video_frames = []
video_side = []

for cap in caps:

    fps = cap.get(cv2.CAP_PROP_FPS)

    times = []
    y_values = []
    frame_numbers = []


    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:


        # Get middle frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_FRAME_COUNT)/2)

        ret, frame = cap.read()

        # Check if left or right 
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Check which ankle is closer to camera
        try:
            world_landmarks = results.pose_world_landmarks.landmark
            left_ankle = [world_landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, world_landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y, world_landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]
            right_ankle = [world_landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, world_landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y, world_landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]

            if left_ankle[2] < right_ankle[2]:
                video_side.append('left')
            else:
                video_side.append('right')

        except:
            print("Could not detect ankle")
            break



        while cap.isOpened():
            ret, frame = cap.read()

            if not ret:
                break

            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)

            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


            # Extract landmarks
            try:
                
                world_landmarks = results.pose_world_landmarks.landmark

                ankle = None

                if video_side[-1] == 'left':
                    ankle = [world_landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, world_landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y, world_landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]
                else:
                    ankle = [world_landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, world_landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y, world_landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]

                # Get time and y value
                time = cap.get(cv2.CAP_PROP_POS_FRAMES) / fps
                times.append(time)
                y_values.append(ankle[1])

                frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
                frame_numbers.append(frame_number)


            except:
                pass

    video_frames.append((frame_numbers, np.array(y_values)))


# Plot the y values for each video on a separate plot

for video in video_frames:
    plt.plot(video[0], video[1])
    plt.ylabel('Y-Coordinate')
    plt.xlabel('Frame Number')
    plt.show()


lowest_y_values = []

for video in video_frames:
    # Find local minima in data, separated by at least 10 frames, ensure peak depth is at least 0.01
    peaks, _ = find_peaks(-video[1], distance=10)
    # Add peaks along with their frame numbers to a list
    peaks_and_frames = []
    for peak in peaks:
        peaks_and_frames.append((video[0][peak], video[1][peak]))

print(peaks_and_frames)
                


        




To understand robustness of model:

o research the influence of the quality of the video on the robustness of the solution, we recorded predictions of our solution for different videos under multiple input quality configurations. For each video we varied the fps value, resolution, and duration. Specifically, the duration parameter is the number of seconds we use to find angles at the lowest pedal points. The fps and resolution parameters are simply a downsampling of the original quality of the input video.

For each configuration we record the values of the angle at the lowest pedal point. To measure the influence of a reduction in quality we look at the standard deviation of the angles. A higher standard deviation for a configuration means that the angles used to make the recommendation are more spread out and thus our recommendation is less precise.

In [ ]:
# Use the frame number to get the lowest y value for each video
# Calculate the angle between the hip, knee and the ankle for each frame
# Only look at the frames where the ankle is at the lowest y value

video_angles = []
i = 0

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    for cap in caps:
        angles = []

        for frame in peaks_and_frames:
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame[0])

            ret, image = cap.read()
            if not ret:
                print("Could not read frame")
                break


            # Recolor image to RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            angle_results = pose.process(image)


            # Recolor back to BGR
            image.flags.writeable = True
            hip = None
            knee = None
            ankle = None

            try:

                world_landmarks = angle_results.pose_world_landmarks.landmark
                if video_side[i] == 'left':
                    hip = [world_landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, world_landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                    knee = [world_landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, world_landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                    ankle = [world_landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, world_landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                else:
                    hip = [world_landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, world_landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                    knee = [world_landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, world_landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                    ankle = [world_landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, world_landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            except:
                print("Could not detect landmarks")
                pass
        

            # Calculate angle
            angle = _calculate_angle(hip, knee, ankle)
            angles.append(angle)
        
        video_angles.append(angles)
            
        i += 1


print(video_angles)






In [ ]:
# Post processing

# Filter out values that are far from median. This is to remove outliers
processed_angles = []
for video in video_angles:
    median = np.median(video)
    filtered = [x for x in video if abs(x - median) < 10]
    processed_angles.append(filtered)

print(processed_angles)

# However, any one angle might still contain some noise. For example, we cannot observe the position at the true lowest point because that point might lie in between two different frames. 
# The mean of all the observed angles should be a good estimate of the angle at the true lowest point. We use this mean angle for our final recommendation.


# Calculate the mean angle for each video
mean_angles = []
for video in processed_angles:
    mean_angles.append(np.mean(video))

print(mean_angles)